# Code generation for barycentric coordinates

This notebook consists of two main parts, both used in `ocellaris/utils/taylor_basis.py` for Lagrange to Taylor and Taylor to Lagrange transformations for discontinuous Galerkin shape functions on simplices.

**1: Code generation for quadratic barycentric shape functions**

Given a triangle/tetrahedron with barycentric coordinates, find the linear and quadratic shape functions. We here work only in barycentric coordinates. The linear basis functions should be the coordinates themselves and the interesting part is the quadratic shape functions

**2: Code generation for the cartesian to barycentric transformation**

Given cartesian coordinates of all cell vertices, compute the barycentric coordinate transformation for an arbitrary point inside the cell. This is done for 2D (fast) and 3D (dog slow)

In [1]:
import time
from IPython.display import display, HTML
import numpy
from matplotlib import pyplot
import sympy
from sympy import *
sympy.init_printing(use_latex=True)
%matplotlib inline

## Barycentric basis functions in 2D

In [2]:
barycentric_basis = {}

for P in (1, 2):
    display(HTML('<b>Barycentric basis in 2D of order %d</b>' % P))
    D = 2
    N = D + 1
    M = N*P

    # Define the barycentric coordinates L
    Ls = list(symbols('L_(1:%d)' % N))
    Ls.append(1 - sum(Ls))

    # Define the coeffcients of the basis functions, C
    Cs = symbols('C_(1:%d)' % (1 + M*M))
    
    # DOF locations
    w = 1/S(2)
    locs = [(1, 0, 0), (0, 1, 0), (0, 0, 1), (0, w, w), (w, 0, w), (w, w, 0)][:M]

    # Create the linear basis
    Bs = []
    for i in range(M):
        cs = Cs[i*M:(i + 1)*M]
        if P == 1:
            Bs.append(cs[0] + cs[1]*Ls[0] + cs[2]*Ls[1])
        else:
            Bs.append(cs[0] + cs[1]*Ls[0] + cs[2]*Ls[1] + cs[3]*Ls[0]**2 + cs[4]*Ls[1]**2 + cs[5]*Ls[0]*Ls[1])

    # Conditions on the basis, Bi(xj) = delta_j(xj)
    eqs = []
    for iloc, loc in enumerate(locs):
        subs = {Ls[i]: loc[i] for i in range(N)}
        for jb in range(M):
            rhs = 1 if iloc == jb else 0
            eqs.append((Bs[jb] - rhs).subs(subs))

    # Find the coefficients of the basis
    sol = solve(eqs, Cs)
    Ln = Symbol('L_%d' % N)
    Lnsubs = {Ls[-1]: Ln}
    Bsol = []
    for i in range(M):
        Bi = Bs[i]
        Bi = factor(Bi).subs(Lnsubs).simplify().subs(Lnsubs)
        Bi = Bi.subs(sol)
        Bi = factor(Bi).subs(Lnsubs).simplify().subs(Lnsubs)
        Bi = Bi.subs(2*Ls[-1] - 1, 2*Ln -1)
        Bsol.append(Bi.simplify())
        display(Eq(Symbol('B_%d' % i), Bsol[-1]))
    
    barycentric_basis[(D, P)] = Bsol, Ls[:-1] + [Ln]


## Barycentric basis functions in 3D

In [3]:
for P in (1, 2):
    display(HTML('<b>Barycentric basis in 3D of order %d</b>' % P))
    D = 3
    N = D + 1
    M = 4 if P == 1 else 10

    # Define the barycentric coordinates L
    Ls = list(symbols('L_(1:%d)' % N))
    Ls.append(1 - sum(Ls))

    # Define the coeffcients of the basis functions, C
    Cs = symbols('C_(1:%d)' % (1 + M*M))
    
    # DOF locations
    w = 1/S(2)
    locs = [(1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0), (0, 0, 0, 1), (0, 0, w, w),
            (0, w, 0, w), (0, w, w, 0), (w, 0, 0, w), (w, 0, w, 0), (w, w, 0, 0)][:M]

    # Create the linear basis
    Bs = []
    for i in range(M):
        cs = Cs[i*M:(i + 1)*M]
        if P == 1:
            Bs.append(cs[0] + cs[1]*Ls[0] + cs[2]*Ls[1] + cs[3]*Ls[2])
        else:
            Bs.append(cs[0] + cs[1]*Ls[0] + cs[2]*Ls[1] + cs[3]*Ls[2] +
                      cs[4]*Ls[0]**2 + cs[5]*Ls[1]**2 + cs[6]*Ls[2]**2 +
                      cs[7]*Ls[0]*Ls[1] + cs[8]*Ls[0]*Ls[2] + cs[9]*Ls[1]*Ls[2])

    # Conditions on the basis, Bi(xj) = delta_j(xj)
    eqs = []
    for iloc, loc in enumerate(locs):
        subs = {Ls[i]: loc[i] for i in range(N)}
        for jb in range(M):
            rhs = 1 if iloc == jb else 0
            eqs.append((Bs[jb] - rhs).subs(subs))

    # Find the coefficients of the basis
    sol = solve(eqs, Cs)
    Ln = Symbol('L_%d' % N)
    Lnsubs = {Ls[-1]: Ln}
    Bsol = []
    for i in range(M):
        Bi = Bs[i]
        Bi = factor(Bi).subs(Lnsubs).simplify().subs(Lnsubs)
        Bi = Bi.subs(sol)
        Bi = factor(Bi).subs(Lnsubs).simplify().subs(Lnsubs)
        Bi = Bi.subs(2*Ls[-1] - 1, 2*Ln -1)
        Bsol.append(Bi.simplify())
        display(Eq(Symbol('B_%d' % i), Bsol[-1]))
    
    barycentric_basis[(D, P)] = Bsol, Ls[:-1] + [Ln]


## Code generation

In [4]:
def tocode(Bi):
    s = str(Bi)
    s = s.replace('L_1', 'L1')
    s = s.replace('L_2', 'L2')
    s = s.replace('L_3', 'L3')
    s = s.replace('L_4', 'L4')
    return s

for k, (Bs, Ls) in barycentric_basis.items():
    display(HTML('<b>Code for %dD, order %d' % k))
    s = '\n'.join('phi[%d] = %s' % (i, tocode(Bi)) for i, Bi in enumerate(Bs))
    display(HTML('<pre>' + s + '</pre>'))

## Compute barycentric coordinates from cartesian vertex coordinates

In [5]:
barycentric_transform = {}

xs = x1, x2, x3, x4 = symbols('x(1:5)', real=True)
ys = y1, y2, y3, y4 = symbols('y(1:5)', real=True)
zs = z1, z2, z3, z4 = symbols('z(1:5)', real=True)
x, y, z = symbols('x y z', real=True)

for D in (2, 3):
    display(HTML('<h3>Cartesian to barycentric %dD</h3>' % D))
    t1 = time.time()
    N = D + 1
    
    # Coefficients
    Cs = symbols('C_(:%d)' % (N*D))

    # Barycentric coordinates
    if D == 2:
        Ls = [Cs[0] + Cs[1]*x + Cs[2]*y,
              Cs[3] + Cs[4]*x + Cs[5]*y]
        Ls.append(1 - Ls[0] - Ls[1])
    else:
        Ls = [Cs[0] + Cs[1]*x + Cs[2]*y + Cs[3]*z,
              Cs[4] + Cs[5]*x + Cs[6]*y + Cs[7]*z,
              Cs[8] + Cs[9]*x + Cs[10]*y + Cs[11]*z,]
        Ls.append(1 - Ls[0] - Ls[1] - Ls[2])

    # Build equation system
    eqs = []
    # Loop over corners, i
    for i in range(N):
        vertex_pos = {x: xs[i], y: ys[i], z: zs[i]}
        # Loop over barycentric coordinates, j
        for j in range(N-1):
            rhs = 1 if i == j else 0
            eqs.append(Ls[j].subs(vertex_pos) - rhs)

    display(Eq(Symbol('N_\\text{eq}'), len(eqs)))
    display(Eq(Symbol('N_\\text{Cs}'), len(Cs)))

    # Solve the equation system
    display(HTML('<b style="color:#999900">This is going to take some time, especially in 3D ...</b>'))
    sol = solve(eqs, Cs)
    
    F = Symbol('F')
    if D == 2:
        fac = x1*y2 - x1*y3 - x2*y1 + x2*y3 + x3*y1 - x3*y2
        #diff = fac - (y2 - y3)*(x1 - x3) + (x3 - x2)*(y1 - y3)
        #display(expand(diff).simplify())
    else:
        fac = x1*y2*z3 - x1*y2*z4 - x1*y3*z2 + x1*y3*z4 + x1*y4*z2 - x1*y4*z3 - x2*y1*z3 + x2*y1*z4 + x2*y3*z1 - x2*y3*z4 - x2*y4*z1 + x2*y4*z3 + x3*y1*z2 - x3*y1*z4 - x3*y2*z1 + x3*y2*z4 + x3*y4*z1 - x3*y4*z2 - x4*y1*z2 + x4*y1*z3 + x4*y2*z1 - x4*y2*z3 - x4*y3*z1 + x4*y3*z2
    display(Eq(F, factor(fac)))
    
    # Show the solution
    Lsol = []
    for i in range(N):
        Li = (Ls[i] * fac).subs(sol).simplify()
        Lsol.append(Li/F)
        display(Eq(Symbol('L_%d' % i), factor(Lsol[-1])))
    
    barycentric_transform[D] = Lsol
    display(HTML('<b style="color:#009900">Done in %.2f seconds</b>' % (time.time() - t1)))


N_\text{eq} = 6

N_\text{Cs} = 6

N_\text{eq} = 12

N_\text{Cs} = 12

## Code generation

In [6]:
def tocode(Li):
    return str(Li)

for D, Ls in barycentric_transform.items():
    display(HTML('<b>Code for %dD' % D))
    s = '\n'.join('L%df = %s' % (i + 1, tocode(Li)) for i, Li in enumerate(Ls))
    display(HTML('<pre>' + s + '</pre>'))


### Test implementation against known answers

In [7]:
# First we check for linearity
cell = {x1: 1, y1: 1, z1: 1,
        x2: 2, y2: 1, z2: 2,
        x3: 4, y3: 2, z3: 3,
        x4: 7, y4: 2, z4: 4}
Fs = {2: x1*y2 - x1*y3 - x2*y1 + x2*y3 + x3*y1 - x3*y2,
      3: x1*y2*z3 - x1*y2*z4 - x1*y3*z2 + x1*y3*z4 + x1*y4*z2 - x1*y4*z3 - x2*y1*z3 + x2*y1*z4 + x2*y3*z1 - x2*y3*z4 - x2*y4*z1 + x2*y4*z3 + x3*y1*z2 - x3*y1*z4 - x3*y2*z1 + x3*y2*z4 + x3*y4*z1 - x3*y4*z2 - x4*y1*z2 + x4*y1*z3 + x4*y2*z1 - x4*y2*z3 - x4*y3*z1 + x4*y3*z2}
for D, Ls in barycentric_transform.items():
    display(HTML('<b>Code for %dD' % D))
    display(Eq(Symbol('F'), Fs[D].subs(cell)))
    for i, L in enumerate(Ls):
        LF = (L*F).subs(cell).subs(F, Fs[D].subs(cell)).simplify()
        display(Eq(Symbol('L_{%d}\\,F' % i), LF))

In [8]:
def test_cell_2D(vertices):
    print('Checking cell', vertices, '...', end=' ')
    x1, y1 = vertices[0]
    x2, y2 = vertices[1]
    x3, y3 = vertices[2]
    
    def our(x, y):
        "Our solution"
        F = x1*y2 - x1*y3 - x2*y1 + x2*y3 + x3*y1 - x3*y2
        L1f = (x*(y2 - y3) + x2*y3 - x3*y2 - y*(x2 - x3))/F
        L2f = (-x*(y1 - y3) - x1*y3 + x3*y1 + y*(x1 - x3))/F
        L3f = (x*y1 - x*y2 - x1*y + x1*y2 + x2*y - x2*y1)/F
        return L1f, L2f, L3f
    
    def ref(x, y):
        D = (y2 - y3)*(x1 - x3) + (x3 - x2)*(y1 - y3)
        L1f = ((y2 - y3)*(x - x3) + (x3 - x2)*(y - y3))/D
        L2f = ((y3 - y1)*(x - x3) + (x1 - x3)*(y - y3))/D
        L3f = 1 - L1f - L2f
        return L1f, L2f, L3f
    
    # Generate some positions to check
    other_positions = []
    for A, B in [(1/3, 1/3), (1/2, 1/4), (1/4, 1/2),
                 (3/5, 1/5), (1/5, 3/5), (1/5, 1/5)]:
        C = 1 - A - B
        other_positions.append((A*x1 + B*x2 + C*x3,
                                A*y1 + B*y2 + C*y3))
    
    OK = True
    for v in vertices + other_positions:
        #print('Checking', v)
        o1, o2, o3 = our(v[0], v[1])
        r1, r2, r3 = ref(v[0], v[1])
        for o, r in zip((o1, o2, o3), (r1, r2, r3)):
            if not abs(o - r) < 1e-15:
                print(o, '-', r, '=', o-r, '   <-- ERROR!!!')
                OK = False
    
    print('OK' if OK else 'ERROR')

print('Testing 2D cells')
test_cell_2D([(0, 0), (1, 0), (0, 1)])
test_cell_2D([(1, 1), (1, 0), (0, 1)])
test_cell_2D([(cell[x1], cell[y1]),
              (cell[x2], cell[y2]),
              (cell[x3], cell[y3])])
test_cell_2D([(-3, 4), (8, -1), (-15, -1)])

def test_cell_3D(vertices):
    print('Checking cell', vertices, '...', end=' ')
    x1, y1, z1 = vertices[0]
    x2, y2, z2 = vertices[1]
    x3, y3, z3 = vertices[2]
    x4, y4, z4 = vertices[3]
    
    def our(x, y, z):
        "Our solution"
        F = x1*y2*z3 - x1*y2*z4 - x1*y3*z2 + x1*y3*z4 + x1*y4*z2 - x1*y4*z3 - x2*y1*z3 + x2*y1*z4 + x2*y3*z1 - x2*y3*z4 - x2*y4*z1 + x2*y4*z3 + x3*y1*z2 - x3*y1*z4 - x3*y2*z1 + x3*y2*z4 + x3*y4*z1 - x3*y4*z2 - x4*y1*z2 + x4*y1*z3 + x4*y2*z1 - x4*y2*z3 - x4*y3*z1 + x4*y3*z2
        L1f = (x*(y2*z3 - y2*z4 - y3*z2 + y3*z4 + y4*z2 - y4*z3) - x2*y3*z4 + x2*y4*z3 + x3*y2*z4 - x3*y4*z2 - x4*y2*z3 + x4*y3*z2 - y*(x2*z3 - x2*z4 - x3*z2 + x3*z4 + x4*z2 - x4*z3) + z*(x2*y3 - x2*y4 - x3*y2 + x3*y4 + x4*y2 - x4*y3))/F
        L2f = (-x*(y1*z3 - y1*z4 - y3*z1 + y3*z4 + y4*z1 - y4*z3) + x1*y3*z4 - x1*y4*z3 - x3*y1*z4 + x3*y4*z1 + x4*y1*z3 - x4*y3*z1 + y*(x1*z3 - x1*z4 - x3*z1 + x3*z4 + x4*z1 - x4*z3) - z*(x1*y3 - x1*y4 - x3*y1 + x3*y4 + x4*y1 - x4*y3))/F
        L3f = (x*(y1*z2 - y1*z4 - y2*z1 + y2*z4 + y4*z1 - y4*z2) - x1*y2*z4 + x1*y4*z2 + x2*y1*z4 - x2*y4*z1 - x4*y1*z2 + x4*y2*z1 - y*(x1*z2 - x1*z4 - x2*z1 + x2*z4 + x4*z1 - x4*z2) + z*(x1*y2 - x1*y4 - x2*y1 + x2*y4 + x4*y1 - x4*y2))/F
        L4f = (-x*y1*z2 + x*y1*z3 + x*y2*z1 - x*y2*z3 - x*y3*z1 + x*y3*z2 + x1*y*z2 - x1*y*z3 - x1*y2*z + x1*y2*z3 + x1*y3*z - x1*y3*z2 - x2*y*z1 + x2*y*z3 + x2*y1*z - x2*y1*z3 - x2*y3*z + x2*y3*z1 + x3*y*z1 - x3*y*z2 - x3*y1*z + x3*y1*z2 + x3*y2*z - x3*y2*z1)/F
        return L1f, L2f, L3f, L4f
    
    def ref(x, y, z):
        if (x, y, z) == vertices[0]:
            return 1, 0, 0, 0
        elif (x, y, z) == vertices[1]:
            return 0, 1, 0, 0
        elif (x, y, z) == vertices[2]:
            return 0, 0, 1, 0
        elif (x, y, z) == vertices[3]:
            return 0, 0, 0, 1
        elif (x, y, z) == other_positions[0]: # centroid
            return 1/4, 1/4, 1/4, 1/4
    
    ## Generate some positions to check
    other_positions = []
    for A, B, C in [(1/4, 1/4, 1/4)]:
        D = 1 - A - B - C
        other_positions.append((A*x1 + B*x2 + C*x3 + D*x4,
                                A*y1 + B*y2 + C*y3 + D*y4,
                                A*z1 + B*z2 + C*z3 + D*z4))
    
    OK = True
    for v in vertices + other_positions:
        #print('Checking', v)
        os = our(v[0], v[1], v[2])
        rs = ref(v[0], v[1], v[2])
        for i in range(4):
            o, r = os[i], rs[i]
            #print('    %d: % .2f - % .2f = % .2f' % (i, o, r, o-r),
            #      '   <-- ERROR!!!' if not abs(o - r) < 1e-15 else '')
            if not abs(o - r) < 1e-15:
                OK = False
    
    print('OK' if OK else 'ERROR')

print('Testing 3D cells')
test_cell_3D([(0, 0, 0), (1, 0, 0), (0, 1, 0), (0, 0, 1)])
test_cell_3D([(0, 0, 0), (1, 0, 0), (0, 1, 0), (1, 1, 1)])
test_cell_3D([(cell[x1], cell[y1], cell[z1]),
              (cell[x2], cell[y2], cell[z2]),
              (cell[x3], cell[y3], cell[z3]),
              (cell[x4], cell[y4], cell[z4])])
test_cell_3D([(-4, 2, 33), (-2, -1, -50), (-10, 1, -30), (55, 22, -30)])

Testing 2D cells
Checking cell [(0, 0), (1, 0), (0, 1)] ... OK
Checking cell [(1, 1), (1, 0), (0, 1)] ... OK
Checking cell [(1, 1), (2, 1), (4, 2)] ... OK
Checking cell [(-3, 4), (8, -1), (-15, -1)] ... OK
Testing 3D cells
Checking cell [(0, 0, 0), (1, 0, 0), (0, 1, 0), (0, 0, 1)] ... OK
Checking cell [(0, 0, 0), (1, 0, 0), (0, 1, 0), (1, 1, 1)] ... OK
Checking cell [(1, 1, 1), (2, 1, 2), (4, 2, 3), (7, 2, 4)] ... OK
Checking cell [(-4, 2, 33), (-2, -1, -50), (-10, 1, -30), (55, 22, -30)] ... OK
